# Série temporelle : décomposition par méthode de Buys Ballot et approximation de la perturbation par un modèle AR

In [ ]:
import pandas as pd # pour les séries temporelles
import matplotlib.pyplot as plt # pour les plot
import numpy as np
from copy import deepcopy
from BuysBallot import BuysBallotModel  # module python pour la méthode de Buys Ballot
from datetime import datetime  # gestion des dates
import scipy.stats  # outils statistiques
import statsmodels.api as sm # outils pour les séries temporelles

## Lecture des données

In [ ]:
path = "Data/ukcp09_gridded-land-obs-daily_timeseries_mean-temperature_000000E_500000N_19600101-20161231.csv"
data = pd.read_csv(path, header=[0,1], index_col=0, parse_dates=True)

In [ ]:
data.head()

In [ ]:
# Suppression des relevés pour les 29/02 des années bissextiles
data = pd.read_csv(path, header=[0,1], index_col=0, parse_dates=True)
dates_to_remove = [datetime.date(datetime.strptime(str(y)+"-02-29", '%Y-%m-%d')) for y in range(1960, 2017, 4)]
data = data.drop(dates_to_remove)

# sélection d'une série temporelle (un lieu)
series = data["22500"]["547500"]
sup = len(series)-1
inf = 0
dates = data.index

# plot
plt.figure(figsize=(10,10))
plt.plot(list(series[:365*5]))
plt.title("Températures entre le 01/01/1960 et le 31/12/1964 (54.7500°N -2.2500°W)")
plt.show()

## Décomposition par méthode de Buys Ballot

In [ ]:
# Définition des fonctions de base pour la régression
def f2(t, i, j, length, subd = 1):
    return np.power(t,i) * (t>=j*length/subd).astype(int)*(t<(j+1)*length/subd).astype(int)


def g(t, i, k = 365):
    return (t%k == i).astype(int) - (t%k == 0).astype(int)

In [ ]:
# Méthode de Buys Ballot
BBM = BuysBallotModel(f2, g, subd=1, dim=2)

In [ ]:
BBM.plot_decomposition(data,"22500", "547500", save=False)

## Autocorrélation et autocorrélation partielle

In [ ]:
# Lecture de la série des résidus (enregistrée indépendemment)
serie = pd.read_csv("BB22500-547500.csv", header=None)[1]
serie.index = data.index
serie.head()

In [ ]:
# Autocorrélation et autocorrélation partielle
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(serie, lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(serie, lags=40, ax=ax2)

## Estimation des paramètres du modèle AR

In [ ]:
# Modèle ARMA
model1 = sm.tsa.ARMA(serie, (4,0))
result1 = model1.fit(disp=False)

In [ ]:
result1.params

In [ ]:
confidence = result1.conf_int(alpha=0.05)
print((confidence[1]-confidence[0])/2)

In [ ]:
model1.loglike(result1.params)

In [ ]:
result1.pvalues

In [ ]:
result1.plot_predict(20750,20800)
plt.show()

## Validation du modèle

In [ ]:
# Résidus -> bruit blanc ?
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(result1.resid, lags=40, ax=ax1, alpha=0.05)

In [ ]:
# Normalité de la distribution des résidus ?
normalized_resid = result1.resid/np.std(result1.resid)
x = np.linspace(-4,4,200)
plt.figure(figsize=(10,10))
plt.hist(normalized_resid.values, bins = int((2*len(normalized_resid))**(1/3)), normed=True, label="Distribution empirique")
plt.plot(x, scipy.stats.norm.pdf(x), color = 'r', label = "N(0,1)")
plt.legend()
plt.show()

In [ ]:
# Test de Kolmogorov-Smirnov
r = scipy.stats.kstest(normalized_resid, "norm")
r